In [ ]:
from pygraphblas import *
from pygraphblas.base import _check, lib
import pygraphblas.descriptor
import csv
import sys
import logging
import glob
import operator
from data_loader import DataLoader

In [ ]:
# Setup logger
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-5s %(message)s'))
log = logging.getLogger(__name__)
log.propagate = False
log.addHandler(handler)
log.setLevel(logging.INFO)

In [ ]:
data_dir = 'sf1k-converted/'
loader = DataLoader(data_dir)

vertices, mappings, matrices = loader.load_all_csvs()

In [ ]:
with open(data_dir + 'person.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='|', quotechar='"')
    birthdayColName, = [colName for colName in reader.fieldnames if colName.split(':')[0]=='birthday']
    personBirthdays = [row[birthdayColName] for row in reader]
    

In [ ]:
topK=3
birthdayLimit='1986-06-14'

isPersonSelected = [birthday>=birthdayLimit for birthday in personBirthdays]
birthdayPersonMask = Vector.from_list(isPersonSelected)
birthdayPersonMask.select(lib.GxB_NONZERO, out=birthdayPersonMask)
print(f'Selected persons: {birthdayPersonMask.nvals}')

hasInterestTranMx=matrices['hasInterest'].transpose()
for tagCol in range(len(vertices['tag'])):
    personVec = hasInterestTranMx[tagCol]
    personVec *= birthdayPersonMask
    
    personColsInSubgraph = [colValPair[0] for colValPair in personVec]
    
    subgraphMx = matrices['knows'][personColsInSubgraph, personColsInSubgraph]
    print(subgraphMx)
    
    out = ffi.new('GrB_Vector*')
    _check(lib.LAGraph_cc_fastsv(
            out,
            subgraphMx.matrix[0],
            False))
    
    new_type = ffi.new('GrB_Type*')
    _check(lib.GxB_Vector_type(new_type, out[0]))
    result= Vector(out, types.gb_type_to_type(new_type[0]))
    print(result)
    print(result.to_string())
    
    if subgraphMx.nvals:
        break
